Notes to try: Feature importance; conversion of data to 0 and 1; logistic regression threshold

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import  cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_curve,log_loss, precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.metrics import make_scorer
import seaborn as sns
import matplotlib.pyplot as plt
import itertools


In [2]:
dfraw = pd.read_csv('https://raw.githubusercontent.com/robholmstrom/Sams_github/master/Cap_2/patient.csv')

HTTPError: HTTP Error 404: Not Found

In [ ]:
dfraw.shape

In [ ]:
pd.set_option("display.max_rows", 101)

In [ ]:
dfraw.head(3)

In [ ]:
dfraw.info()

In [ ]:
dfraw.columns

# Data cleaning

In [3]:
# DO NOT ALTER THESE LISTS!!!!!

# Select out continous variable in seperate list
unaltercols = ['age','outcome']
# Select all columns that  will be treated the same way
altercols = ['sex', 'patient_type', 'intubated', 'pneumonia',  'pregnant',
       'diabetes', 'copd', 'asthma', 'immunosuppression', 'hypertension',
       'other_diseases', 'cardiovascular', 'obesity', 'chronic_kidney_failure',
       'smoker', 'icu']

# Eliminate unclears (98 values), clear nulls (99)  and convert non-applicable (97) to 'no' for all columns
# except pregnant or icu columns.
# This will allow more choice downstream.
dfreplace = pd.concat([dfraw[unaltercols], dfraw[altercols].replace({99:np.nan, 97:2, 98:np.nan})], axis = 1)

# Further eliminate nulls from pregnant column
#dfreplace1 = pd.concat([dfreplace,dfraw['pregnant'].replace(98,np.nan)], axis = 1)                    

# Further eliminate nulls from icu column
#dfreplace2 = pd.concat([dfreplace, dfraw['icu'].replace(99, np.nan)],axis=1)
dfreplace2 = dfreplace.loc[dfreplace['patient_type'] == 2,:]
dfreplace2 = pd.get_dummies(dfreplace2, columns = ['outcome']).drop('patient_type', axis = 1)
dfreplace2.columns

NameError: name 'dfraw' is not defined

In [4]:
dfreplace2.info()

NameError: name 'dfreplace2' is not defined

# Feature engineering

In [5]:
# In case pregnancy is as feature, the following code builds two dataframes to choose from: dfpreg or dfall 
target = 'intubated'
class_balance = 'yes'


dfpreg = dfreplace2.loc[(dfreplace2['age']>=dfreplace2.loc[dfreplace2['pregnant']==1]['age'].min()) & (dfreplace2['age']<=dfreplace2.loc[dfreplace2['pregnant']==1]['age'].max()) & (dfreplace2['sex'] == 1)]
if 'sex' in dfpreg: dfpreg = dfpreg.drop('sex', axis = 1).drop('icu', axis = 1)

dficu = dfreplace2.loc[(dfreplace2['icu']==1)].drop('icu', axis = 1)

dfall = dfreplace2.copy().drop('icu', axis = 1)

custom_feat_cols = ['age', 'pneumonia', 'immunosuppression', 'other_diseases', 'obesity',
       'outcome_1', 'outcome_2']





  

dfdropcols = [x for x in dfall if x != target and x not in custom_feat_cols]

dfdropcols

NameError: name 'dfreplace2' is not defined

In [6]:

dfpregdrop= pd.concat([dfall['sex'], dfpreg], axis = 1).drop(dfdropcols, axis = 1)
dfalldrop= dfall.drop(dfdropcols, axis = 1)
dficudrop= pd.concat([dficu], axis = 1).drop(dfdropcols, axis = 1)

# Choose df here:

df = dficudrop



NameError: name 'dfall' is not defined

In [7]:
if target == 'icu': 
  df['icu'] = dfreplace2['icu']
  if 'intubated' in df:
    df = df.drop('intubated', axis = 1)

In [8]:
#Convert all 2s to 0 as they represent 'no' and 1 represents yes, except age, if in columns

if 'age' in df:
    columns_for_conversion = df.columns.tolist()
    columns_for_conversion.remove('age')
    df[columns_for_conversion] = df.loc[:,columns_for_conversion].replace(2,0)
else:
    df.replace(2,0, inplace=True)
    



NameError: name 'df' is not defined

In [9]:
# Deleting leftover nulls from conversion of 99 and 98 to NaN
df_non_nulls = df.dropna()

# Checking nulls have been eliminated
df_non_nulls.info()

NameError: name 'df' is not defined

In [10]:
# Investigate cross-feature correlations to eliminate multicolinearity
df_non_nulls.corr()

NameError: name 'df_non_nulls' is not defined

In [11]:
# Seperate features and target for modeling
X = df_non_nulls.drop(target, axis=1)
y = df_non_nulls[target]

NameError: name 'df_non_nulls' is not defined

In [12]:
X.shape

NameError: name 'X' is not defined

In [13]:
k = 7
if len(X.columns) >= k:
    skb = SelectKBest(k = k, score_func = f_classif)
    best_kfit = skb.fit(X, y)
    X_best_feat = best_kfit.fit_transform(X,y)

    X_best_feat = pd.DataFrame(X_best_feat, columns=X.columns[best_kfit.get_support()])

    print(X_best_feat.columns)
else:
    pass

NameError: name 'X' is not defined

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

NameError: name 'X' is not defined

In [15]:
if class_balance == 'yes':
    from sklearn.utils import resample
    X1 = pd.concat([X_train,y_train], axis=1)

    not_target = X1[X1[target]==0]
    targetsam = X1[X1[target]==1]
    upsample_target = resample(targetsam,replace=True, n_samples=len(not_target))
    upsampled = pd.concat([not_target, upsample_target])
    y_train = upsampled[target]
    X_train = upsampled.drop(target, axis=1)
else:
    pass

NameError: name 'X_train' is not defined

In [16]:
print(len(X_train), len(y_train))

NameError: name 'X_train' is not defined

In [17]:
sns.countplot(y_train)
plt.title('Class Balance')
plt.show()


NameError: name 'y_train' is not defined

# Logistic Regressions

Class imbalance will pose a problem for predicting instances of the less frequent category e.g intubated

In [18]:
lr1 = LogisticRegression()
thresholds = [x for x in np.arange(.01,1,.01)]
c_values = [10**x for x in range(-3,2)]
params = {'solver':['liblinear','lbfgs'], 'C':c_values, 'penalty':['l2'], 'max_iter': [10000]}

gslr1 = GridSearchCV(lr1, params)
gslr1.fit(X_train, y_train)


print(gslr1.best_params_)
    
y_test_predlr1 = gslr1.predict(X_test)
y_train_predlr1 = gslr1.predict(X_train)


y_train_probs = gslr1.predict_proba(X_train)
y_test_probs = gslr1.predict_proba(X_test)

test_intubation_scores = []
train_intubation_scores = []
y_pred_test_thresh = []
y_pred_train_thresh = []
for i in thresholds:
    
    y_pred_test_thresh = [1 if x[1] >= i else 0 for x in y_test_probs]
    test_intubation_scores.append(recall_score(y_test, y_pred_test_thresh, average = 'macro'))
    
    y_pred_train_thresh = [1 if x[1] >= i else 0 for x in y_train_probs]
    train_intubation_scores.append(recall_score(y_train, y_pred_train_thresh, average = 'macro'))

best_test_thresh = thresholds[test_intubation_scores.index(max(test_intubation_scores))]
best_train_thresh = thresholds[train_intubation_scores.index(max(train_intubation_scores))]

y_pred_test = [1 if x[1] >= best_test_thresh else 0 for x in y_test_probs]   
y_pred_train = [1 if x[1] >= best_train_thresh else 0 for x in y_train_probs]

sns.lineplot(thresholds, test_intubation_scores)


print(f'Best threshold for test macro average recall score for {target} is ', best_test_thresh)



NameError: name 'X_train' is not defined

In [19]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=None):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    
   

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')
        
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black", fontsize=20)
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",fontsize=20)


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [20]:
from sklearn.metrics import confusion_matrix
print('LR1 confusion matrix')
lr1train = confusion_matrix(y_train, y_pred_train, normalize= None)
lr1test = confusion_matrix(y_test, y_pred_test, normalize = None)


LR1 confusion matrix


NameError: name 'y_train' is not defined

In [21]:
target_names = ['not '+target, target]
plot_confusion_matrix(lr1test,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=None)

NameError: name 'lr1test' is not defined

In [22]:
print('Test macro average recall score is ',recall_score(y_test, y_pred_test, average = 'macro'))
print('Train macro average recall score is ', recall_score(y_train, y_pred_train,average = 'macro'))

NameError: name 'y_test' is not defined

In [23]:
                                         
lr1best = LogisticRegression(**gslr1.best_params_)
lr1best.fit(X_train, y_train)
abs_coefs = np.abs(lr1best.coef_[0])
coefs = pd.DataFrame(abs_coefs, index=X_train.columns, columns=['Coef'])
coefs.loc['INTERCEPT'] = [lr1best.intercept_[0]]
coefs.sort_values('Coef', ascending=False)


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

# Random Forests

In [24]:

rf1 = RandomForestClassifier(max_depth=3,  max_features=4,  n_estimators=10000)
rf1.fit(X_train, y_train)

y_test_predrf1 = rf1.predict(X_test)
y_train_predrf1 = rf1.predict(X_train)
print('Test macro average recall score is ', recall_score(y_test, y_test_predrf1, average = 'macro'))
print('Train macro average recall score is ', recall_score(y_train, y_train_predrf1,average = 'macro'))


NameError: name 'X_train' is not defined

In [25]:
print('RF1 confusion matrix')
print('Training\n', confusion_matrix(y_train, y_train_predrf1, normalize= None))
print('Test\n', confusion_matrix(y_test, y_test_predrf1, normalize = None))

RF1 confusion matrix


NameError: name 'y_train' is not defined

# K Nearest Neighbors

In [26]:

knn1 = KNeighborsClassifier(n_neighbors=6, weights='uniform', leaf_size=20)
knn1.fit(X_train, y_train)

y_test_predknn1 = knn1.predict(X_test)
y_train_predknn1 = knn1.predict(X_train)
print('Test macro average recall score is ',recall_score(y_test, y_test_predknn1,average = 'macro'))
print('Train macro average recall score is ',recall_score(y_train, y_train_predknn1,average = 'macro'))


NameError: name 'X_train' is not defined

In [27]:

print(f'KNN1 cv scores are {cross_val_score(knn1, X_train, y_train, cv= 5)}')



NameError: name 'X_train' is not defined

In [28]:
print('knn1 confusion matrix')
print('Training\n', confusion_matrix(y_train, y_train_predknn1, normalize= None))
print('Test\n', confusion_matrix(y_test, y_test_predknn1, normalize = None))

knn1 confusion matrix


NameError: name 'y_train' is not defined

# Support Vector Classification

In [29]:
c_values = [10**x for x in range(-2, 0)]
svmlin = SVC()
parameters = {'kernel':['linear'], 'C':c_values}
svmgrid = GridSearchCV(svmlin, parameters)
svmgrid.fit(X_train, y_train)

y_train_predsvmgrid = svmgrid.predict(X_train)
y_test_predsvmgrid = svmgrid.predict(X_test)

print('Test macro average recall score is ', recall_score(y_test, y_test_predsvmgrid,average = 'macro'))
print('Train macro average recall score is ', recall_score(y_train, y_train_predsvmgrid,average = 'macro'))
print(svmgrid.best_params_)


NameError: name 'X_train' is not defined

In [30]:

print('svmrbf confusion matrix')
print('Training\n', confusion_matrix(y_train, y_train_predsvmgrid, normalize= None))
print('Test\n', confusion_matrix(y_test,y_test_predsvmgrid, normalize = None))


svmrbf confusion matrix


NameError: name 'y_train' is not defined

In [31]:
print(f'SVMlin cv scores are {cross_val_score(svmlin, X, y, cv= 5)}')

NameError: name 'X' is not defined